# Notebook d'importation des données de Wikidata

ce notebook sert de base pour la création des différentes tables utiles pour la base de données. il est structuré en fonction des données nécessaire à mon analyse

# import des librairies

In [207]:
### Librairie à installer dans l'environnement conda
# qui sera choisi pour exécuter le carnet
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Librairies déjà installées avec Python
import pprint
import csv
import sys

import sqlite3 as sql

import time
import datetime
from dateutil import parser

from importlib import reload
from shutil import copyfile

### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
# sys.path.insert(0, '../..')

import sparql_functions as spqf

print("Les imports se sont bien passés")

Les imports se sont bien passés


## table des personnes et des partis

Déroulement de l'exécution du notebook:
1. Récupérer les personnes avec les proprités nécessaire
2. Nettoyer les données (eg. annee_naissance doit être un integer et pas une chaîne de caractères, créer les codes)
3. Construire les liste à partir de la liste des personnes en dataframe
4. Créer les différentes tables à partir des dataframes y relatif:
5. on se connecte à la base de données pour récupérer la table `wdt_personnes`
6. Créer la table des personne avec les contraintes fk_lieu_naissance_id, fk_genre_id et fk_spatialisation_id. et on insert les personnes avec la méthode `to_sql` du dataframe mais **avec l'option `if_exists='drop table'`**.
```sql
CREATE TABLE wdt_personnes ( 
 id INTEGER PRIMARY KEY, 
 personne_uri TEXT,
 nom_prenom TEXT,
 annee_naissance INTEGER,
 fk_spatialisation_id INTEGER,
 fk_lieu_naissance_id INTEGER,
 fk_genre_id INTEGER,
 FOREIGN KEY (fk_lieu_naissance_id) REFERENCES wdt_lieu_naissance(id),
 FOREIGN KEY (fk_spatialisation_id) REFERENCES wdt_spatialisation(id),
 FOREIGN KEY (fk_genre_id) REFERENCES wdt_genre(id)
);
   ```
7. on insère les autres tables dans la table `wdt_personnes` via des fk, ainsi qu'une suppression des doublons
8. création des tables pivots `wdt_personnes_partis` et `wdt_personnes_langues`

# étape 1 - import des personnes dans un dataframe

In [208]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [209]:
query = """
SELECT DISTINCT ?item  ?itemLabel  ?gender ?birthYear ?partiUri ?langues ?languesLabel ?lieunaissance ?lieunaissanceLabel ?coordinates # ?type
        WHERE {
            {
         
          {?item wdt:P39 wd:Q18510612} #p39 position held et Q18510612 pour conseil national
            UNION
            {?item wdt:P17 wd:Q39} # p17 country et Q39 switzerland
         
              }
          
          ?item wdt:P31 wd:Q5;  # Any instance of a human.
              wdt:P569 ?birthDate;
                wdt:P21 ?gender;
                wdt:P102 ?partiUri;
                wdt:P1412 ?langues;
                wdt:P19 ?lieunaissance.
         OPTIONAL {
            ?lieunaissance wdt:P625 ?coordinates.
              # ?birthPlace  wdt:P31 ?type
          }
        BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?birthYear)
        FILTER(xsd:integer(?birthYear) > 1900 )

          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        } 
     ORDER BY ?birthYear
"""

print ("u nail it")

u nail it


In [210]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

print ("tt est bon")

tt est bon


In [211]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

1123


[['http://www.wikidata.org/entity/Q120229',
  'Robert Tobler',
  'http://www.wikidata.org/entity/Q6581097',
  '1901',
  'http://www.wikidata.org/entity/Q672926',
  'http://www.wikidata.org/entity/Q279350',
  'Standard High German',
  'http://www.wikidata.org/entity/Q72',
  'Zürich',
  'Point(8.541111111 47.374444444)'],
 ['http://www.wikidata.org/entity/Q120229',
  'Robert Tobler',
  'http://www.wikidata.org/entity/Q6581097',
  '1901',
  'http://www.wikidata.org/entity/Q684909',
  'http://www.wikidata.org/entity/Q279350',
  'Standard High German',
  'http://www.wikidata.org/entity/Q72',
  'Zürich',
  'Point(8.541111111 47.374444444)'],
 ['http://www.wikidata.org/entity/Q120229',
  'Robert Tobler',
  'http://www.wikidata.org/entity/Q6581097',
  '1901',
  'http://www.wikidata.org/entity/Q13142801',
  'http://www.wikidata.org/entity/Q279350',
  'Standard High German',
  'http://www.wikidata.org/entity/Q72',
  'Zürich',
  'Point(8.541111111 47.374444444)']]

In [212]:
### Créer un DataFrame à partir du résultat
df_personnes = pd.DataFrame(r)
df_personnes.columns = ['personne_uri', 'nom_prenom', 'genre_uri', 'annee_naissance', 'parti_uri', 'langue_uri', 'langue', 'lieu_naissance_uri', 'lieu_naissance', 'lieu_coord']
df_personnes[df_personnes['nom_prenom'] == 'Robert Tobler'].head(100)

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,langue_uri,langue,lieu_naissance_uri,lieu_naissance,lieu_coord
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q672926,http://www.wikidata.org/entity/Q279350,Standard High German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444)
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q684909,http://www.wikidata.org/entity/Q279350,Standard High German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444)
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q13142801,http://www.wikidata.org/entity/Q279350,Standard High German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444)
3,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q672926,http://www.wikidata.org/entity/Q387066,Swiss German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444)
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q684909,http://www.wikidata.org/entity/Q387066,Swiss German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444)
5,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q13142801,http://www.wikidata.org/entity/Q387066,Swiss German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444)


In [213]:
print(df_personnes.dtypes)

personne_uri          object
nom_prenom            object
genre_uri             object
annee_naissance       object
parti_uri             object
langue_uri            object
langue                object
lieu_naissance_uri    object
lieu_naissance        object
lieu_coord            object
dtype: object


# étape 2 - nettoyage des données

## transformer les années de naissance de str à int

In [214]:
def annee_naissance_int(annee_naissance):
    return int(annee_naissance)

### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['annee_naissance'] = df_personnes.apply(lambda x: annee_naissance_int(x['annee_naissance']), axis=1)

In [215]:
print(df_personnes.dtypes)

personne_uri          object
nom_prenom            object
genre_uri             object
annee_naissance        int64
parti_uri             object
langue_uri            object
langue                object
lieu_naissance_uri    object
lieu_naissance        object
lieu_coord            object
dtype: object


## création du genCode

In [216]:
 def code_genre(genre_uri):
    if genre_uri == 'http://www.wikidata.org/entity/Q6581097':
        # genre masculin
        a = 'M'
    elif genre_uri =='http://www.wikidata.org/entity/Q6581072':
        # genre féminin
        a = 'F'
    else :
        return 'A'
    return a  
     
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['genre_code'] = df_personnes.apply(lambda x: code_genre(x['genre_uri']), axis=1)

In [217]:
### On inspecte les cinq premières lignes qui ont la valeur 'F' = féminin
df_personnes[df_personnes['genre_code']=='F'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,langue_uri,langue,lieu_naissance_uri,lieu_naissance,lieu_coord,genre_code
101,http://www.wikidata.org/entity/Q1515691,Gertrud Spiess,http://www.wikidata.org/entity/Q6581072,1914,http://www.wikidata.org/entity/Q659461,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q78,Basel,Point(7.590555555 47.560555555),F
116,http://www.wikidata.org/entity/Q1903085,Martha Ribi,http://www.wikidata.org/entity/Q6581072,1915,http://www.wikidata.org/entity/Q13850128,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),F
182,http://www.wikidata.org/entity/Q1944817,Monique Bauer-Lagier,http://www.wikidata.org/entity/Q6581072,1922,http://www.wikidata.org/entity/Q667785,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q68230,Meyrin,Point(6.07911 46.23222),F
186,http://www.wikidata.org/entity/Q1329797,Elisabeth Blunschy,http://www.wikidata.org/entity/Q6581072,1922,http://www.wikidata.org/entity/Q659461,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q68155,Schwyz,Point(8.658332 47.020546),F
187,http://www.wikidata.org/entity/Q1329797,Elisabeth Blunschy,http://www.wikidata.org/entity/Q6581072,1922,http://www.wikidata.org/entity/Q659461,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q68155,Schwyz,Point(8.658332 47.020546),F


In [218]:
print(df_personnes.dtypes)

personne_uri          object
nom_prenom            object
genre_uri             object
annee_naissance        int64
parti_uri             object
langue_uri            object
langue                object
lieu_naissance_uri    object
lieu_naissance        object
lieu_coord            object
genre_code            object
dtype: object


## création du code_parti

In [219]:
def code_parti(parti_uri):
    if parti_uri == 'http://www.wikidata.org/entity/Q303745':
        # PS
        a = 'PS'
    elif parti_uri =='http://www.wikidata.org/entity/Q385258':
        # UDC
        a = 'UDC'
    elif parti_uri =='http://www.wikidata.org/entity/Q13850128':
        # PRD
        a = 'PRD'
    elif parti_uri =='http://www.wikidata.org/entity/Q659461':
        # PDC
        a = 'PRD'
    elif parti_uri =='http://www.wikidata.org/entity/Q659739':
        # Les Verts
        a = 'Verts'
    elif parti_uri =='http://www.wikidata.org/entity/Q202638':
        # PLR
        a = 'PLR'
    elif parti_uri =='http://www.wikidata.org/entity/Q667785':
        # PLS
        a = 'PLS'
    elif parti_uri =='http://www.wikidata.org/entity/Q550295':
        # alliance des indépendants
        a = 'ALL_IND'
    elif parti_uri =='http://www.wikidata.org/entity/Q545900':
        # PVL
        a = 'PVL'
    elif parti_uri =='http://www.wikidata.org/entity/Q100934025':
        # le centre
        a = 'LC'
    elif parti_uri =='http://www.wikidata.org/entity/Q151768':
        # parti bourgeois démocrate
        a = 'PBD'
    elif parti_uri =='http://www.wikidata.org/entity/Q477228':
        # parti des paysans et artisans indépendants
        a = 'PAI'
    elif parti_uri =='http://www.wikidata.org/entity/Q667718':
        # parti évangélique suisse
        a = 'PEV'
    elif parti_uri =='http://www.wikidata.org/entity/Q661771':
        # parti suisse du travail
        a = 'PST'
    elif parti_uri =='http://www.wikidata.org/entity/Q660046':
        # démocrates suisse ou action nationale
        a = 'DS'
    elif parti_uri =='http://www.wikidata.org/entity/Q668101':
        # ligue des Tessinois
        a = 'lega'
    elif parti_uri =='http://www.wikidata.org/entity/Q2145393':
        # mouvement républicain
        a = 'MR'
    elif parti_uri =='http://www.wikidata.org/entity/Q667836':
        # union démocratique fédérale
        a = 'UDF'
    elif parti_uri =='http://www.wikidata.org/entity/Q681843':
        # parti démocrate suisse
        a = 'PDS'
    elif parti_uri =='http://www.wikidata.org/entity/Q667725':
        # parti chrétien social
        a = 'PDC'
    elif parti_uri =='http://www.wikidata.org/entity/Q663580':
        # mouvement citoyens genevois
        a = 'MCG'
    elif parti_uri =='http://www.wikidata.org/entity/Q362949':
        # organisation progressistes de Suisse
        a = 'POCH'
    elif parti_uri =='http://www.wikidata.org/entity/Q327591':
        # indépendant
        a = 'IND'
    elif parti_uri =='http://www.wikidata.org/entity/Q13124':
        # parti libéral démocrate
        a = 'FDP'
    elif parti_uri =='http://www.wikidata.org/entity/Q684909':
        # collection fédérale Eidgenössische Sammlung
        a = 'CF'
    elif parti_uri =='http://www.wikidata.org/entity/Q677491':
        # mouvement socio-libéral
        a = 'SLB'
    elif parti_uri =='http://www.wikidata.org/entity/Q672946':
        # parti suisse de la liberté
        a = 'PSL'
    elif parti_uri =='http://www.wikidata.org/entity/Q672926':
        # front national
        a = 'FN'
    elif parti_uri =='http://www.wikidata.org/entity/Q3366732':
        # parti socialiste autonome
        a = 'PSA'
    elif parti_uri =='http://www.wikidata.org/entity/Q3366188':
        # parti chrétien-social indépendant
        a = 'PCSI'
    elif parti_uri =='http://www.wikidata.org/entity/Q286683':
        # solidaritéS
        a = 'Sol'
    elif parti_uri =='http://www.wikidata.org/entity/Q255650':
        # la Gauche alternative linke
        a = 'AL'
    elif parti_uri =='http://www.wikidata.org/entity/Q20012603':
        # ensemble à gauche
        a = 'EaG'
    elif parti_uri =='http://www.wikidata.org/entity/Q13142801':
        # Neue Front
        a = 'NeF'
    elif parti_uri =='http://www.wikidata.org/entity/Q1254507':
        # alliance verte et sociale bernoise, grünes bündnis
        a = 'GB'
    elif parti_uri =='http://www.wikidata.org/entity/Q1186111':
        # demokratisch-soziale partei
        a = 'PDC'
    elif parti_uri =='http://www.wikidata.org/entity/Q1084149':
        # parti chrétien-social d'obwald
        a = 'PCS'
    elif parti_uri =='http://www.wikidata.org/entity/Q10594788':
        # action nationale pour le peuple et la patrie
        a = 'NA'
    else:
        return 'A'
        
    return a 

### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['code_parti'] = df_personnes.apply(lambda x: code_parti(x['parti_uri']), axis=1)

In [220]:
### On inspecte les cinq premières lignes qui ont la valeur 'PAI' = xxx
df_personnes[df_personnes['code_parti']=='PAI'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,langue_uri,langue,lieu_naissance_uri,lieu_naissance,lieu_coord,genre_code,code_parti
14,http://www.wikidata.org/entity/Q125303,Dewet Buri,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q477228,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q10317,Fraubrunnen,Point(7.526944444 47.085),M,PAI
31,http://www.wikidata.org/entity/Q2543940,Walter Degen,http://www.wikidata.org/entity/Q6581097,1904,http://www.wikidata.org/entity/Q477228,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q64735,Bennwil,Point(7.77987 47.40317),M,PAI
50,http://www.wikidata.org/entity/Q2173417,Rudolf Meier,http://www.wikidata.org/entity/Q6581097,1907,http://www.wikidata.org/entity/Q477228,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q69175,Eglisau,Point(8.52607 47.57335),M,PAI
73,http://www.wikidata.org/entity/Q2039809,Otto Locher,http://www.wikidata.org/entity/Q6581097,1910,http://www.wikidata.org/entity/Q477228,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q67066,Hasle bei Burgdorf,Point(7.655555555 47.017777777),M,PAI
84,http://www.wikidata.org/entity/Q16101728,Otto Bretscher,http://www.wikidata.org/entity/Q6581097,1911,http://www.wikidata.org/entity/Q477228,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q68423,Andelfingen,Point(8.678888888 47.594722222),M,PAI


## création code langues

In [221]:
## ce code_langues permet de déterminer 3 langues et de regrouper toutes les mentions 
## de suisse allemand ou high standard german sous la catégorie 'allemand'

def code_langue(langue_uri):
    if langue_uri == 'http://www.wikidata.org/entity/Q652':
        # italien
        a = 'IT'
    elif langue_uri =='http://www.wikidata.org/entity/Q150':
        # français
        a = 'FR'
    elif langue_uri =='http://www.wikidata.org/entity/Q188	':
        # allemand
        a = 'ALL'
    else :
        return 'ALL'
    return a  
     
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['code_langue'] = df_personnes.apply(lambda x: code_langue(x['langue_uri']), axis=1)

In [222]:
### On inspecte les cinq premières lignes qui ont la valeur 'FR' = français
df_personnes[df_personnes['code_langue']=='FR'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,langue_uri,langue,lieu_naissance_uri,lieu_naissance,lieu_coord,genre_code,code_parti,code_langue
13,http://www.wikidata.org/entity/Q121118,Gabriel Despland,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q13850128,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q70586,Lussery-Villars,Point(6.533333333 46.633333333),M,PRD,FR
15,http://www.wikidata.org/entity/Q15621580,Jacques Chamorel,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q667785,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q70115,Ollon,Point(6.99348 46.29731),M,PLS,FR
17,http://www.wikidata.org/entity/Q116107,Jean Bourgknecht,http://www.wikidata.org/entity/Q6581097,1902,http://www.wikidata.org/entity/Q659461,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q36378,Fribourg,Point(7.15 46.8),M,PRD,FR
19,http://www.wikidata.org/entity/Q119847,Jean Peitrequin,http://www.wikidata.org/entity/Q6581097,1902,http://www.wikidata.org/entity/Q13850128,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q807,Lausanne,Point(6.633333333 46.533333333),M,PRD,FR
21,http://www.wikidata.org/entity/Q23636365,René Villard,http://www.wikidata.org/entity/Q6581097,1902,http://www.wikidata.org/entity/Q303745,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q1034,Biel/Bienne,Point(7.247222222 47.137222222),M,PS,FR


## création code_lieu

> Besoin de séparer les données entre ville-campagne-étranger, comme les URI seront différents, il me semble mieux de faire un code manuellement.

In [223]:
## ce code_langues permet de déterminer 3 langues et de regrouper toutes les mentions 
## de suisse allemand ou high standard german sous la catégorie 'allemand'

def code_lieu(lieu_naissance_uri):
    if lieu_naissance_uri == 'http://www.wikidata.org/entity/Q72':
        # zurich
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70115':
        # ollon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7133	':
        # grosswangen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69126':
        # möhlin
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64982':
        # fahrwangen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q40120':
        # luino
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68015':
        # roggwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70586':
        # Lussery-Villars
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q10317':
        # fraubrunnen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q182806':
        # näfels
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68129':
        # olten
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q36378':
        # fribourg
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q807':
        # lausanne
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q1034':
        # bienne
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q78':
        # bâle
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q490':
        # milan
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69345':
        # neuchâtel
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65699':
        # Tschappina
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68277':
        # Buchs
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q4191':
        # lucerne
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70515':
        # rivaz
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67499':
        # herzogebuchsee
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68097':
        # davos
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64735':
        # bennwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69111':
        # schleitheim
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67048':
        # grabs
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70090':
        # saint-croix
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7024':
        # lugano
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63979':
        # Versoix
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69191':
        # laufenburg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68160':
        # vevey
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69535':
        # icogne
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q25607':
        # st-gall
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63946':
        # yverdon-les-bains
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65125':
        # romanshorn
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14547':
        # entlebuch
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69175':
        # eglisau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67523':
        # biasca
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68124':
        # chx
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q43236':
        # avenches
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q486284':
        # heerbrugg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q13209':
        # nidau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7131':
        # nottwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q9125':
        # winterhur
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66688':
        # bolken
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q2841':
        # Bogota
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q661652':
        # engi
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q9009':
        # schaffouse
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68144':
        # zug
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q43195':
        # aigle
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q43195':
        # Gachnang
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q71':
        # genève
        a = 'V'    
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66775':
        # erschwil
        a = 'C'   
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q2546289':
        # plainpalais
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q27576':
        # wettingen
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67066':
        # hasle bei burgdorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69160':
        # bolligen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66819':
        # rüschlikon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66411':
        # niederweningen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66442':
        # wölflinswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68423':
        # andelfingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67044':
        # muolen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q455496':
        # höngg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64239':
        # staffelbach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70':
        # bern
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66841':
        # bannwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65415':
        # erlinsbach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68155':
        # schwyz
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67501':
        # quinto
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66763':
        # Rechthalten
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67282':
        # urtenen-schönbühl
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69216':
        # künsacht
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65993':
        # schafisheim
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64156':
        # arbon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69172':
        # hallau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63912':
        # glaris
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63912':
        # Wald
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64533':
        # kienberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q28587':
        # hamadan
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66298':
        # walchwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69085':
        # Samedan
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q209270':
        # appenzell
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69429':
        # lengnau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69786':
        # raron
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68256':
        # porrentruy
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q90':
        # paris
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69639':
        # göschenen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69858':
        # schwadernau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67544':
        # mallerey
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q2596602':
        # wüflingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67489':
        # wiler
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7083':
        # rothburg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68752':
        # mathod
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14584':
        # alberswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66142':
        # deitingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66908':
        # escholzmatt
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68347':
        # disentis
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69635':
        # wassen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q659932':
        # luchsingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q11935':
        # locarno
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68308':
        # bulle
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65775':
        # mosnang
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q662294':
        # hauteville
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69011':
        # muri
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69673':
        # faido
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64719':
        # laufelfingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65924':
        # kappelen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67450':
        # Busswil bei Melchnau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65649':
        # courtelary
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67048':
        # grabs
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15815120':
        # hennens
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68230':
        # meyrin
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q52300':
        # bussigny
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65309':
        # menziken
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66390':
        # st-imier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q818824':
        # maroua
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q661454':
        # linthal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69665':
        # fluelen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68429':
        # Bossonnens
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65158':
        # stetten
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69595':
        # rheineck
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66781':
        # gaiserwald
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66996':
        # oberhof
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64957':
        # jonen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64961':
        # felsberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65381':
        # baldingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64191':
        # walenstadt
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64077':
        # wil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14628':
        # zell
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14587':
        # dagmersellen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65981':
        # Krauchthal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q589968':
        # wengen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63986':
        # zofingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64147':
        # visp/viège
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69765':
        # muri bei bern
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68286':
        # horgen
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70432':
        # hauterive
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66788':
        # wartau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65455':
        # balsthal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64990':
        # Oberentfelden
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67157':
        # moutier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69774':
        # stafa
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70517':
        # st-aubin sauges
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64269':
        # untersiggenthal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68899':
        # liddes
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64433':
        # Rottenschwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67666':
        # saignelégier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64250':
        # wiliberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14274':
        # aarau
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68250':
        # rorschach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7138':
        # triengen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15819953':
        # illhart
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q3572':
        # tunis
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64032':
        # uster
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q239':
        # bruxelles
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70078':
        # peseux
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66060':
        # Hauenstein-Ifenthal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69445':
        # vufflens le chateau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15582':
        # brig
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69007':
        # chur
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7149':
        # neudorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63970':
        # herisau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63896':
        # delemont
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66228':
        # quarten
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65684':
        # sagogn
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q52694':
        # champvent
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q1380511':
        # glosersville
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q52697':
        # Chapelle-sur-Moudon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67058':
        # Uetikon am See
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15789302':
        # bonau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64457':
        # birmenstorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69654':
        # Feusisberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68978':
        # thun
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66718':
        # leuggern
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68220':
        # oftringen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66087':
        # häggenschil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65093':
        # schluein
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65577':
        # roggwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69507':
        # boudry
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q5465':
        # cap town
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69309':
        # la rippe
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69995':
        # sorengo
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67670':
        # lumino
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68445':
        # bure
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q668912':
        # chabrey
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64113':
        # aarberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70675':
        # pompaples
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q43195':
        # aigle
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67798':
        # plaffein
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69558':
        # savièse
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69961':
        # muralto
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69729':
        # Rapperswil-Jona
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67743':
        # novazzano
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69572':
        # saxon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65200':
        # Hofstetten bei Brienz
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66702':
        # zwingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7123':
        # sursee
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7102':
        # hochdorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64213':
        # ilanz
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7145':
        # pfeffikon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q60':
        # new-york
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68965':
        # solothurn
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64093':
        # locle
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68956':
        # martigny
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66218':
        # kandersteg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69790':
        # cadenazzo
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67318':
        # berg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69079':
        # scuol
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65508':
        # balgach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67993':
        # bonfol
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q689575':
        # gampel
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67238':
        # lauperswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68220':
        # oftringen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64108':
        # engelberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63905':
        # kreuzlingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70141':
        # balerna
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68297':
        # sierre
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65880':
        # belfaux
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7133':
        # grosswangen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q2152101':
        # riedt
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70202':
        # moudon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69726':
        # langenthal
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64920':
        # breil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q2444039':
        # torgon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68471':
        # binn
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q689668':
        # salins
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66300':
        # langnau im emmental
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68281':
        # altstätten
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65506':
        # flawil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65009':
        # niedergösgen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64682':
        # pratval
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69448':
        # le paquier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69436':
        # leukerbad
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69401':
        # morges
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q9093':
        # bülach
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65781':
        # berg im irchel
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q11737':
        # rüti bei büren
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64873':
        # illnau-effretikon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69777':
        # gossau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68063':
        # saint-prex
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70501':
        # Valeyres-sous-Rances
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67215':
        # messen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68290':
        # Affoltern am Albis
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65921':
        # Grossaffoltern
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q1781':
        # budapest
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15241660':
        # rheinklingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64630':
        # waldrirch
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14592':
        # ettiswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69517':
        # chatelet-saint-denis
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63939':
        # baden
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64687':
        # titterten
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q660142':
        # hofen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q793951':
        # azmoos
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69256':
        # wilderswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65957':
        # diemtigen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65658':
        # unterägeri
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q52743':
        # le chenit
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64027':
        # nyon
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7151':
        # sempach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66246':
        # vilters-wangs
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68305':
        # wetzikon
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69296':
        # alpnach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q52699':
        # chateau d'oex
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q546':
        # trieste
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14557':
        # romoos
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70169':
        # lalden
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68120':
        # frauenfeld
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65455':
        # balsthal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q48320':
        # kobe
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q220':
        # rome
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66069':
        # muotathal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65446':
        # schanïs
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64171':
        # uzwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67499':
        # Herzogenbuchsee
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69235':
        # cham
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q869624':
        # mogelsberg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65492':
        # wattenwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68167':
        # sumiswald
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64482':
        # rothrist
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65732':
        # Sigriswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66705':
        # bassecourt
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65102':
        # donat
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64752':
        # aristau
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68286':
        # horgen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69671':
        # gordola
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65071':
        # Mümliswil-Ramiswil 
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q22690':
        # Metz
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66949':
        # alterswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69738':
        # pully
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69678':
        # belp
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68950':
        # einsieden
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68460':
        # stalden
        a = 'C'
    elif lieu_naissance_uri =='http://http://www.wikidata.org/entity/Q69619':
        # huttwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7135':
        # oberkirch
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q406':
        # istanbul
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66868':
        # Eggersriet
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67644':
        # rüderswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66635':
        # Schattdorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q63964':
        # sarnen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14559':
        # Schüpfheim
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64087':
        # baar
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67097':
        # unterseen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68136':
        # sion
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68311':
        # burgdorf
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q37149':
        # gstaad
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7124':
        # RUSWIL
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14571':
        # kriens
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14577':
        # Udligenswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q52298':
        # bursins
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q4225':
        # emmen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q8717':
        # seville
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70385':
        # auvergnier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67217':
        # lüsslingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68191':
        # bardonnex
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70418':
        # châtonnaye
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64099':
        # brugg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q79815':
        # mulhouse
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69579':
        # Grimisuat
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68787':
        # sessa
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66580':
        # galgenen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q661499':
        # castasegna
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64044':
        # bellinzona
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q3130':
        # sydney
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q216972':
        # Uíge
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66554':
        # sissach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64755':
        # Merenschwand
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69374':
        # wallisellen
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70317':
        # salgesch
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69233':
        # Münsterlingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66601':
        # wattwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68959':
        # thalwil
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q11939':
        # Stein am Rhein
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70503':
        # lignerolle
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14570':
        # horw
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q628354':
        # fleurier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q1780':
        # bratislava
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68248':
        # granges
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70708':
        # Schwarzenburg
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66208':
        # Frutigen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7144':
        # rickenbach
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64627':
        # Männedorf
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q2254412':
        # schwarzsee
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67145':
        # bauma
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q117181':
        # elbistan
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14552':
        # Flühli
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q689582':
        # La Tour-de-Trême
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q648':
        # Lilles
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66132':
        # Erlenbach im Simmental
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q132404':
        # bourges
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70557':
        # Denezy
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69530':
        # Chêne-Bougeries
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q1335':
        # Montevideo
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66532':
        # bubendorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69290':
        # génolier
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q12146':
        # basel-lndschaft
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68210':
        # Adliswil
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q1486':
        # buenos aires
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67951':
        # Dielsdorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66884':
        # Schönenwerd
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64051':
        # monthey
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68150':
        # altdorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15788657':
        # billens
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q136217':
        # erzincan
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69819':
        # Dürrenroth
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70133':
        # comano
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67679':
        # monteggio
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q10341':
        # jegenstorf
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64840':
        # gachnang
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q67272':
        # wald
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q7136':
        # mauensee
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64178':
        # maienfeld
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65120':
        # häfelfingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68972':
        # liestal
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q14555':
        # hasle
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q65286':
        # hägglingen
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69718':
        # wädenswil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q31803316':
        # Staad SG	
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q66066':
        # wolfwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q64912':
        # roveredo
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q340':
        # montreal
        a = 'E'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q69619':
        # huttwil
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q15730054':
        # glaris
        a = 'V'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q68972':
        # liestal
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q29015165':
        # ilanz
        a = 'C'
    elif lieu_naissance_uri =='http://www.wikidata.org/entity/Q70355':
        # randogne
        a = 'C'
    else :
        return 'ALL'
    return a  
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['code_lieu'] = df_personnes.apply(lambda x: code_lieu(x['lieu_naissance_uri']), axis=1)

In [224]:
### On inspecte les cinq premières lignes qui ont la valeur 'FR' = français
df_personnes[df_personnes['code_lieu']=='E'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,langue_uri,langue,lieu_naissance_uri,lieu_naissance,lieu_coord,genre_code,code_parti,code_langue,code_lieu
10,http://www.wikidata.org/entity/Q1290496,Eduard Zellweger,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q303745,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q40120,Luino,Point(8.75 46.0),M,PS,ALL,E
20,http://www.wikidata.org/entity/Q1586125,Mario Agustoni,http://www.wikidata.org/entity/Q6581097,1902,http://www.wikidata.org/entity/Q13850128,http://www.wikidata.org/entity/Q652,Italian,http://www.wikidata.org/entity/Q490,Milan,Point(9.19 45.466944444),M,PRD,IT,E
59,http://www.wikidata.org/entity/Q1892932,Marcel Beck,http://www.wikidata.org/entity/Q6581097,1908,http://www.wikidata.org/entity/Q681843,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q2841,Bogotá,Point(-74.08175 4.60971),M,PDS,ALL,E
118,http://www.wikidata.org/entity/Q3087758,Friedrich Salzmann,http://www.wikidata.org/entity/Q6581097,1915,http://www.wikidata.org/entity/Q550295,http://www.wikidata.org/entity/Q188,German,http://www.wikidata.org/entity/Q28587,Hamadan,Point(48.516666666 34.8),M,ALL_IND,ALL,E
129,http://www.wikidata.org/entity/Q2847509,André Marc Jean Chavanne,http://www.wikidata.org/entity/Q6581097,1916,http://www.wikidata.org/entity/Q303745,http://www.wikidata.org/entity/Q150,French,http://www.wikidata.org/entity/Q90,Paris,Point(2.352222222 48.856666666),M,PS,FR,E


# étape 3A - Construire la liste des partis à partir de la liste des personnes

In [225]:
dict_partis = dict()

for index, row in df_personnes.iterrows():
    if row['code_parti'] not in dict_partis:
        dict_partis[row['code_parti']] = row['parti_uri']

df_partis = pd.DataFrame.from_dict({'parti_code': dict_partis.keys(), 'parti_uri': dict_partis.values()})

print(df_partis)

   parti_code                                  parti_uri
0          FN     http://www.wikidata.org/entity/Q672926
1          CF     http://www.wikidata.org/entity/Q684909
2         NeF   http://www.wikidata.org/entity/Q13142801
3         PRD     http://www.wikidata.org/entity/Q659461
4          PS     http://www.wikidata.org/entity/Q303745
5         PAI     http://www.wikidata.org/entity/Q477228
6         PLS     http://www.wikidata.org/entity/Q667785
7     ALL_IND     http://www.wikidata.org/entity/Q550295
8         PDS     http://www.wikidata.org/entity/Q681843
9          DS     http://www.wikidata.org/entity/Q660046
10        UDC     http://www.wikidata.org/entity/Q385258
11         MR    http://www.wikidata.org/entity/Q2145393
12        PST     http://www.wikidata.org/entity/Q661771
13        FDP      http://www.wikidata.org/entity/Q13124
14        PEV     http://www.wikidata.org/entity/Q667718
15      Verts     http://www.wikidata.org/entity/Q659739
16        PBD     http://www.wi

# étape 3B - Construire la liste des langues à partir de la liste des personnes

In [226]:
dict_langues = dict()

for index, row in df_personnes.iterrows():
    if row['code_langue'] not in dict_langues:
        dict_langues[row['code_langue']] = row['langue_uri']

df_langues = pd.DataFrame.from_dict({'code_langue': dict_langues.keys(), 'langue_uri': dict_langues.values()})

print(df_langues)

  code_langue                              langue_uri
0         ALL  http://www.wikidata.org/entity/Q279350
1          FR     http://www.wikidata.org/entity/Q150
2          IT     http://www.wikidata.org/entity/Q652


# étape 3C - Construire la liste des lieux de naissance à partir de la liste des personnes

In [227]:
dict_lieux = dict()

for index, row in df_personnes.iterrows():
    if row['code_lieu'] not in dict_lieux:
        dict_lieux[row['code_lieu']] = (row['lieu_naissance_uri'])

# Création du DataFrame à partir du dictionnaire
df_lieux = pd.DataFrame.from_dict({'code_lieu': dict_lieux.keys(), 'lieu_naissance_uri': dict_lieux.values()})

print(df_lieux)

  code_lieu                     lieu_naissance_uri
0         V     http://www.wikidata.org/entity/Q72
1         C   http://www.wikidata.org/entity/Q7133
2         E  http://www.wikidata.org/entity/Q40120


## étape 3D - construire la liste spatialisation à partir de la liste des personnes

In [228]:
# Initialisation du dictionnaire
dict_spatialisation = dict()

# Remplissage du dictionnaire avec des valeurs uniques
for index, row in df_personnes.iterrows():
    if row['lieu_coord'] not in dict_spatialisation:
        dict_spatialisation[row['lieu_coord']] = (row['lieu_naissance_uri'], row['lieu_naissance'], row['code_lieu'])

# Création du DataFrame à partir du dictionnaire
df_spatialisation = pd.DataFrame([(k, v[0], v[1], v[2]) for k, v in dict_spatialisation.items()], columns=['lieu_coord', 'lieu_naissance_uri', 'lieu_naissance', 'code_lieu'])


print(df_spatialisation)

                           lieu_coord  \
0     Point(8.541111111 47.374444444)   
1            Point(8.05 47.133055555)   
2     Point(9.062777777 47.098888888)   
3     Point(7.907777777 47.353055555)   
4     Point(7.845833333 47.558333333)   
..                                ...   
372               Point(9.206 46.775)   
373  Point(39.491388888 39.746388888)   
374              Point(7.791 47.0892)   
375          Point(8.95 46.033333333)   
376   Point(7.506944444 47.049722222)   

                           lieu_naissance_uri lieu_naissance code_lieu  
0          http://www.wikidata.org/entity/Q72         Zürich         V  
1        http://www.wikidata.org/entity/Q7133    Grosswangen         C  
2      http://www.wikidata.org/entity/Q182806         Näfels         C  
3       http://www.wikidata.org/entity/Q68129          Olten         V  
4       http://www.wikidata.org/entity/Q69126         Möhlin         C  
..                                        ...            ...       ..

## étape 3E - construire la liste genre à partir de la liste des personnes

In [229]:
# Initialisation du dictionnaire
dict_genres = dict()

# Remplissage du dictionnaire avec des valeurs uniques
for index, row in df_personnes.iterrows():
    if row['genre_code'] not in dict_genres:
        dict_genres[row['genre_code']] = (row['genre_uri'])

# Création du DataFrame à partir du dictionnaire
df_genre = pd.DataFrame.from_dict({'genre_code': dict_genres.keys(), 'genre_uri': dict_genres.values()})

print(df_genre)

  genre_code                                genre_uri
0          M  http://www.wikidata.org/entity/Q6581097
1          F  http://www.wikidata.org/entity/Q6581072


# étape 4 - Créer les table à partir des dataframe

In [230]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/conseil_national_import.db')
cn

# étape 4A - Créer la table des partis à partir du dataframe des partis

In [231]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_partis"
cur = cn.cursor()
cur.execute(query)

In [232]:
query = """
CREATE TABLE wdt_partis ( 
 id INTEGER PRIMARY KEY, 
 parti_code TEXT,
 parti_uri TEXT
)
 """

cur = cn.cursor()
cur.execute(query)

In [233]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_partis.to_sql(name='wdt_partis', con=cn, if_exists='replace', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 4B - Créer la table des langues à partir du dataframe des langues

In [234]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_langues"
cur = cn.cursor()
cur.execute(query)

In [235]:
query = """
CREATE TABLE wdt_langues ( 
 id INTEGER PRIMARY KEY, 
 code_langue TEXT
)
 """

cur = cn.cursor()
cur.execute(query)

In [236]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_langues.to_sql(name='wdt_langues', con=cn, if_exists='replace', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 4C - Créer la table des lieux de naissance à partir du dataframe des lieux de naissance

In [237]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_lieu_naissance"
cur = cn.cursor()
cur.execute(query)

In [238]:
query = """
CREATE TABLE IF NOT EXISTS wdt_lieu_naissance ( 
 id INTEGER PRIMARY KEY, 
 code_lieu TEXT,
 lieu_naissance_uri TEXT
)
 """

cur = cn.cursor()
cur.execute(query)

In [239]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_lieux.to_sql(name='wdt_lieu_naissance', con=cn, if_exists='replace', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 4D - Créer la table spatialisation à partir du dataframe spatialisation

In [240]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_spatialisation"
cur = cn.cursor()
cur.execute(query)

In [241]:
# Création de la table
query = """
CREATE TABLE IF NOT EXISTS wdt_spatialisation ( 
    id INTEGER PRIMARY KEY, 
    lieu_coord TEXT,
    lieu_naissance_uri TEXT,
    lieu_naissance_label TEXT,
    code_lieu TEXT
)
"""

cur = cn.cursor()
cur.execute(query)

In [242]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_spatialisation.to_sql(name='wdt_spatialisation', con=cn, if_exists='replace', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 4E - Créer la table genre à partir du dataframe genres

In [243]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_genre"
cur = cn.cursor()
cur.execute(query)

In [244]:
# Création de la table
query = """
CREATE TABLE IF NOT EXISTS wdt_genre ( 
    id INTEGER PRIMARY KEY, 
    genre_code TEXT,
    genre_uri TEXT
)
"""

cur = cn.cursor()
cur.execute(query)

In [245]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_genre.to_sql(name='wdt_genre', con=cn, if_exists='replace', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 5 - se connecter à la base de données pour prendre la table `wdt_personnes`

In [246]:
import sqlite3

# Connexion à la base de données
cn = sqlite3.connect('../../data/conseil_national_import.db')
cur = cn.cursor()

# Vérification des tables existantes
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Tables existantes :", tables)

# Vérification du contenu de la table wdt_personnes
count = cur.execute("SELECT COUNT(*) FROM wdt_personnes;").fetchone()
print("Nombre de lignes dans wdt_personnes :", count[0])

# Récupération des données de la table wdt_personnes
l = cur.execute("SELECT * FROM wdt_personnes LIMIT 3").fetchall()
print("Données récupérées :", l)


Tables existantes : [('wdt_personnes',), ('wdt_personnes_partis',), ('wdt_personnes_langues',), ('wdt_generations_lieu_naissance',), ('wdt_partis',), ('wdt_langues',), ('wdt_lieu_naissance',), ('wdt_spatialisation',), ('wdt_genre',)]
Nombre de lignes dans wdt_personnes : 885
Données récupérées : [(0, 'http://www.wikidata.org/entity/Q120229', 'Robert Tobler', 1901, 0, 0, 0), (6, 'http://www.wikidata.org/entity/Q1758783', 'Paul Schib', 1901, 1, 1, 0), (7, 'http://www.wikidata.org/entity/Q1579473', 'Hans Fischer', 1901, 2, 1, 0)]


In [247]:
cn = sql.connect('../../data/conseil_national_import.db')
cn

In [248]:
### On récupère tout la table et on la met dans la list 'l'
cur = cn.cursor()
l = cur.execute("SELECT * FROM wdt_personnes limit 3 ").fetchall()

In [249]:
[print(e) for e in l]

(0, 'http://www.wikidata.org/entity/Q120229', 'Robert Tobler', 1901, 0, 0, 0)
(6, 'http://www.wikidata.org/entity/Q1758783', 'Paul Schib', 1901, 1, 1, 0)
(7, 'http://www.wikidata.org/entity/Q1579473', 'Hans Fischer', 1901, 2, 1, 0)


[None, None, None]

# étape 6 - créer la table des personnes avec la contraintes fk_lieu_naissance et fk_spatialisation

In [250]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_personnes"
cur = cn.cursor()
cur.execute(query)

In [251]:
query = """
CREATE TABLE wdt_personnes ( 
 id INTEGER PRIMARY KEY, 
 personne_uri TEXT,
 nom_prenom TEXT,
 annee_naissance INTEGER,
 fk_spatialisation_id INTEGER,
 fk_lieu_naissance_id INTEGER,
 fk_genre_id INTEGER,
 FOREIGN KEY (fk_lieu_naissance_id) REFERENCES wdt_lieu_naissance(id),
 FOREIGN KEY (fk_spatialisation_id) REFERENCES wdt_spatialisation(id),
 FOREIGN KEY (fk_genre_id) REFERENCES wdt_genre(id)
)
 """

cur = cn.cursor()
cur.execute(query)
cn.commit()

# étape 7A- insertion des partis dans la table wdt_personnes

In [252]:
def ajout_fk_parti_id(parti_code):
    ligne_parti = df_partis.index[df_partis['parti_code']== parti_code].tolist()[0]
    return ligne_parti

print(ajout_fk_parti_id('UDC'))

10


# étape 7AB création de la table de correspondance personne -> partis

In [253]:
personnes_partis = dict()

for index, row in df_personnes.iterrows():
    if row['personne_uri'] not in personnes_partis:
        personnes_partis[row['personne_uri']] = set()

    personnes_partis[row['personne_uri']].add(row['code_parti'])

# print(list(x for x in personnes_langues.keys() if x == 'http://www.wikidata.org/entity/Q120229'))
print(personnes_partis['http://www.wikidata.org/entity/Q120229'])

{'FN', 'CF', 'NeF'}


In [254]:
df_personnes = df_personnes.drop(['code_parti', 'parti_uri'], axis=1)

In [255]:
df_personnes.head()

,personne_uri,nom_prenom,genre_uri,annee_naissance,langue_uri,langue,lieu_naissance_uri,lieu_naissance,lieu_coord,genre_code,code_langue,code_lieu
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q279350,Standard High German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,ALL,V
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q279350,Standard High German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,ALL,V
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q279350,Standard High German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,ALL,V
3,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q387066,Swiss German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,ALL,V
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q387066,Swiss German,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,ALL,V


# étape 7B - insertion des langues dans la table wdt_personnes

In [256]:
def ajout_fk_langue_id(code_langue):
    ligne_langue = df_langues.index[df_langues['code_langue']== code_langue].tolist()[0]
    return ligne_langue

print(ajout_fk_langue_id('FR'))

1


In [257]:
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
# df_personnes['fk_langue_id'] = df_personnes.apply(lambda x: ajout_fk_langue_id(x['code_langue']), axis=1)

# étape 7BB création de la table de correspondance personne -> langues

In [258]:
personnes_langues = dict()

for index, row in df_personnes.iterrows():
    if row['personne_uri'] not in personnes_langues:
        personnes_langues[row['personne_uri']] = set()

    personnes_langues[row['personne_uri']].add(row['code_langue'])

# print(list(x for x in personnes_langues.keys() if x == 'http://www.wikidata.org/entity/Q120229'))
print(personnes_langues['http://www.wikidata.org/entity/Q120229'])

{'ALL'}


In [259]:
df_personnes = df_personnes.drop([ 'code_langue', 'langue_uri', 'langue'], axis=1)

In [260]:
df_personnes.head()

,personne_uri,nom_prenom,genre_uri,annee_naissance,lieu_naissance_uri,lieu_naissance,lieu_coord,genre_code,code_lieu
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,V
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,V
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,V
3,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,V
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q72,Zürich,Point(8.541111111 47.374444444),M,V


# étape 7C - insertion des lieux dans la table wdt_personnes

In [261]:
def ajout_fk_lieu_naissance_id(code_lieu):
    ligne_lieu = df_lieux.index[df_lieux['code_lieu']== code_lieu].tolist()[0]
    return ligne_lieu

print(ajout_fk_lieu_naissance_id('V'))

0


In [262]:
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['fk_lieu_naissance_id'] = df_personnes.apply(lambda x: ajout_fk_lieu_naissance_id(x['code_lieu']), axis=1)

In [263]:
df_personnes = df_personnes.drop(['lieu_naissance', 'lieu_naissance_uri', 'code_lieu'], axis=1)

In [264]:
df_personnes.head()

,personne_uri,nom_prenom,genre_uri,annee_naissance,lieu_coord,genre_code,fk_lieu_naissance_id
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,Point(8.541111111 47.374444444),M,0
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,Point(8.541111111 47.374444444),M,0
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,Point(8.541111111 47.374444444),M,0
3,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,Point(8.541111111 47.374444444),M,0
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,Point(8.541111111 47.374444444),M,0


# étape 7D - insertion spatiallisation dans la table wdt_personnes

In [265]:
def ajout_fk_spatialisation_id(lieu_coord):
    ligne_spatialisation = df_spatialisation.index[df_spatialisation['lieu_coord']== lieu_coord].tolist()[0]
    return ligne_spatialisation

print(ajout_fk_spatialisation_id('Point(8.75 46.0)'))

5


In [266]:
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['fk_spatialisation_id'] = df_personnes.apply(lambda x: ajout_fk_spatialisation_id(x['lieu_coord']), axis=1)

In [267]:
df_personnes = df_personnes.drop(['lieu_coord', 'lieu_coord'], axis=1)

In [268]:
df_personnes.head()

,personne_uri,nom_prenom,genre_uri,annee_naissance,genre_code,fk_lieu_naissance_id,fk_spatialisation_id
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,M,0,0
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,M,0,0
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,M,0,0
3,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,M,0,0
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,M,0,0


# étape 7E - insertion genre dans la table wdt_personnes

In [269]:
def ajout_fk_genre_id(genre_code):
    ligne_genre = df_genre.index[df_genre['genre_code']== genre_code].tolist()[0]
    return ligne_genre

print(ajout_fk_genre_id('F'))

1


In [270]:
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['fk_genre_id'] = df_personnes.apply(lambda x: ajout_fk_genre_id(x['genre_code']), axis=1)

In [271]:
df_personnes = df_personnes.drop(['genre_code', 'genre_uri'], axis=1)

In [272]:
df_personnes.head()

,personne_uri,nom_prenom,annee_naissance,fk_lieu_naissance_id,fk_spatialisation_id,fk_genre_id
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,1901,0,0,0
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,1901,0,0,0
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,1901,0,0,0
3,http://www.wikidata.org/entity/Q120229,Robert Tobler,1901,0,0,0
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,1901,0,0,0


# Suppression des doublons

Par example, Robert Tobler parle 2 langues (allemand et suisse allemand) et appartient à 3 partis. Il apparaît donc 6 fois (2 x 3) dans le dataframe. On va donc supprimer ces doublons.

In [273]:
df_personnes = df_personnes.drop_duplicates(subset=['personne_uri'])

In [274]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_personnes.to_sql(name='wdt_personnes', con=cn, if_exists='append', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 8A - Création de la table pivot wdt_personnes_partis

In [275]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_personnes_partis"
cur = cn.cursor()
cur.execute(query)

query = """
CREATE TABLE wdt_personnes_partis ( 
 id INTEGER PRIMARY KEY, 
 fk_personne_id INTEGER,
 fk_parti_id INTEGER,
 FOREIGN KEY (fk_personne_id) REFERENCES wdt_personnes(id),
 FOREIGN KEY (fk_parti_id) REFERENCES wdt_partis(id)
)
 """

cur = cn.cursor()
cur.execute(query)

In [276]:
data = dict({'fk_personne_id': [], 'fk_parti_id': []})

for uri_individu, partis in personnes_partis.items():
    id_individu = df_personnes.index[df_personnes['personne_uri']== uri_individu].tolist()[0]

    for code_parti in partis:
        data['fk_personne_id'].append(id_individu)
        id_parti = df_partis.index[df_partis['parti_code']== code_parti].tolist()[0]
        data['fk_parti_id'].append(id_parti)


        
    
df_personnes_partis = pd.DataFrame(data=data)

In [277]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_personnes_partis.to_sql(name='wdt_personnes_partis', con=cn, if_exists='append', index_label='id')
except Exception as e:
    print('Erreur: ',  e)

# étape 8B - Création de la table pivot wdt_personnes_langues

In [278]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_personnes_langues"
cur = cn.cursor()
cur.execute(query)

query = """
CREATE TABLE wdt_personnes_langues ( 
 id INTEGER PRIMARY KEY, 
 fk_personne_id INTEGER,
 fk_langue_id INTEGER,
 FOREIGN KEY (fk_personne_id) REFERENCES wdt_personnes(id),
 FOREIGN KEY (fk_langue_id) REFERENCES wdt_langues(id)
)
 """

cur = cn.cursor()
cur.execute(query)

In [279]:
data = dict({'fk_personne_id': [], 'fk_langue_id': []})

for uri_individu, langues in personnes_langues.items():
    id_individu = df_personnes.index[df_personnes['personne_uri']== uri_individu].tolist()[0]

    for code_langue in langues:
        data['fk_personne_id'].append(id_individu)
        foo = df_langues.index[df_langues['code_langue']== code_langue].tolist()
        if len(foo) == 0:
            print(id_individu, langues)
            break;
        id_langue = df_langues.index[df_langues['code_langue']== code_langue].tolist()[0]
        data['fk_langue_id'].append(id_langue)


        
    
df_personnes_langues = pd.DataFrame(data=data)

In [280]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_personnes_langues.to_sql(name='wdt_personnes_langues', con=cn, if_exists='append', index_label='id')
except Exception as e:
    print('Erreur: ',  e)